In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from category_encoders import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, RandomForestClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

import warnings
warnings.filterwarnings('ignore')


In [3]:
df = pd.read_csv('./data/train_10pct.csv')
df.head(5)

FileNotFoundError: [Errno 2] No such file or directory: './data/train_10pct.csv'

In [ ]:
df.shape


In [ ]:
df.mean().value_counts(normalize=True)

In [ ]:
df_labels = pd.read_csv('./data/train_10pct_labels.csv')
df_labels.head(5)

In [ ]:
df_labels.shape


In [ ]:
# read in data
train_values = pd.read_csv('../data/Proj5_train_values.csv')
train_labels = pd.read_csv('./data/Proj5_train_labels.csv')
# grab first 10% of rows
train_values = train_values.head(int(len(train_values) * 0.1))
train_labels = train_labels.head(int(len(train_labels) * 0.1))

In [ ]:
print(f'Train value shape - {train_values.shape}')
print(f'Train labels shape - {train_labels.shape}')

In [ ]:
train_values.describe().T

---

## Merge Datasets

In [ ]:
combo = pd.merge(train_values, train_labels, on = 'building_id')
combo.head(5)

In [ ]:
plt.figure (figsize=(5,10));
sns.heatmap(df.corr()[['damage_grade']].sort_values(by='damage_grade', ascending = False),annot=True )

In [ ]:
sns.scatterplot(data=combo, x='age', y='damage_grade', hue='age');

---

In [ ]:
#Baseline
train_labels['damage_grade'].value_counts(normalize=True)

In [ ]:
le = LabelEncoder()
train_enc = train_values.apply(le.fit_transform)
train_enc

In [ ]:
#From Chris

#X = train_enc
#y = trainlabel['damage_grade']
#X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y, random_state=123)
#pipe_forest = make_pipeline(StandardScaler(), DecisionTreeClassifier())
#params = {'decisiontreeclassifier__max_depth' : [2, 3, 4, 5]}
#grid_forest = GridSearchCV(pipe_forest, param_grid = params)
#grid_forest.fit(X_train,y_train)
#grid_forest.score(X_test,y_test) # I got 0.646815042210284
#grid_forest.best_estimator_

In [ ]:
#TTS
X = train_enc
y = train_labels['damage_grade']
#X_train, X_test, y_train, y_test = train_test_split(train_enc,train_labels, random_state=123 )
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y, random_state=123)

## Model

In [ ]:
#from Hackathon2 
#Cvect and logreg
#pipe = make_pipeline(CountVectorizer(stop_words = 'english'), LogisticRegression(n_jobs=-1))
#
#params = {'countvectorizer__max_features':[500, 1000, 15000, 2000, 2500]}
#
#grid=GridSearchCV(pipe, param_grid=params, n_jobs= -1)
#grid.fit(X_train, y_train)
#

### logreg

In [ ]:
#Cvect and logreg
pipe = make_pipeline(StandardScaler(),LogisticRegression(n_jobs=-1))
#
#params = {'countvectorizer__max_features':[500, 1000, 15000, 2000, 2500]}
#
#grid=GridSearchCV(pipe, n_jobs= -1)
pipe.fit(X_train, y_train)
pipe.score(X_train, y_train)

In [ ]:
pipe.fit(X_test, y_test)
pipe.score(X_test, y_test)

In [ ]:
pipe.get_params().keys()


In [ ]:
#LogisticRegression
pipe_lgr = make_pipeline(StandardScaler(), LogisticRegression(n_jobs = -1, max_iter = 1000))
params = {'logisticregression__C' : [0.1, 0.75, 1, 10],
          'logisticregression__solver' : ['newton-cg', 'lbfgs', 'liblinear']}
grid_lgr = GridSearchCV(pipe_lgr, param_grid = params)
grid_lgr.fit(X_train, y_train)
print(f'Train Score: {grid_lgr.score(X_train, y_train)}')
print(f'Test Score: {grid_lgr.score(X_test, y_test)}')
grid_lgr.best_params_

## Modeling KNN

In [ ]:
# define models and parameters
#  model = KNeighborsClassifier()
#  n_neighbors = range(1, 21, 2)
#  weights = ['uniform', 'distance']
#  #metric = ['euclidean', 'manhattan', 'minkowski']
#  metric = ['euclidean']
#  # define grid search
#  grid = dict(n_neighbors=n_neighbors,weights=weights,metric=metric)
#  #cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
#  #grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
#  grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, scoring='accuracy',error_score=0)
#  grid_result = grid_search.fit(X, y)
#  # summarize results
#  print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
#  means = grid_result.cv_results_['mean_test_score']
#  stds = grid_result.cv_results_['std_test_score']
#  params = grid_result.cv_results_['params']
#  for mean, stdev, param in zip(means, stds, params):
#      print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
#Basic KNN
X = train_enc
y = train_labels['damage_grade']
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y, random_state=123)

pipe_knn = make_pipeline(StandardScaler(),KNeighborsClassifier(n_jobs=-1))
pipe_knn.fit(X_train, y_train)
pipe_knn.score(X_train, y_train)
print(f'Train Score: {pipe_knn.score(X_train, y_train)}')
print(f'Test Score: {pipe_knn.score(X_test, y_test)}')

### Trying Veronica's code - KNN testing


In [1]:
pipe_knn = make_pipeline(StandardScaler(), KNeighborsClassifier(n_jobs = -1))

# n_neighbors must be odd to avoid an even split
#Note: tried leaf size and p, but it didn't give us any value
params = {'kneighborsclassifier__n_neighbors' : [5, 7, 9, 11]}  

grid_knn = GridSearchCV(pipe_knn, param_grid = params)
grid_knn.fit(X_train, y_train)
print(f'Train Score: {grid_knn.score(X_train, y_train)}')
print(f'Test Score: {grid_knn.score(X_test, y_test)}')
grid_knn.best_params_

NameError: name 'make_pipeline' is not defined

### 5/13 per Jacob, use OHE instead of LabelEncoding

#### LOG with OHE


In [98]:
X = train_values
y = train_labels['damage_grade']
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y, random_state=123)

---

In [142]:
#Cvect and logreg

#define X and y
X = train_values
y = train_labels['damage_grade']
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y, random_state=123)

#Create pipeline
pipe = make_pipeline(OneHotEncoder(),StandardScaler(with_mean=False), LogisticRegression(n_jobs=-1))

params = {'logisticregression__C' : [0.1, 0.75, 1, 10],
          #'logisticregression__solver' : ['newton-cg', 'lbfgs', 'liblinear']
         }
grid_lgr = GridSearchCV(pipe, param_grid = params)


grid_lgr.fit(X_train, y_train)
grid_lgr.score(X_test, y_test)
print(f'Train Score: {grid_lgr.score(X_train, y_train)}')
print(f'Test Score: {grid_lgr.score(X_test, y_test)}')

Train Score: 0.5925300588385777
Test Score: 0.585111281657713


pandas.core.frame.DataFrame

#### KNN with OHE


In [161]:
#train_values = train_values.head(int(len(train_values) * 0.1))
#train_labels = train_labels.head(int(len(train_labels) * 0.1))
X = train_values
y = train_labels['damage_grade']
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y, random_state=123)

pipe_knn = make_pipeline(OneHotEncoder(),StandardScaler(), KNeighborsClassifier(n_jobs = -1))

# n_neighbors must be odd to avoid an even split
params = {'kneighborsclassifier__n_neighbors' : [5, 7, 9, 11]}
        #'kneighborsclassifier__leaf_size': [1,5,10,30]}


#define parameters for hypertuning
#params = {
#            'n_neighbors': [5, 7, 9, 11],
#            'leaf_size': (1,30),
#            'p': (1,2)

grid_knn = GridSearchCV(pipe_knn, param_grid = params)
grid_knn.fit(X_train, y_train)
print(f'Train Score: {grid_knn.score(X_train, y_train)}')
print(f'Test Score: {grid_knn.score(X_test, y_test)}')
grid_knn.best_params_

Train Score: 0.6720900486057815
Test Score: 0.575287797390637


{'kneighborsclassifier__n_neighbors': 9}

---

In [ ]:
#https://medium.datadriveninvestor.com/k-nearest-neighbors-in-python-hyperparameters-tuning-716734bc557f

#List Hyperparameters that we want to tune.
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]
#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
#Create new KNN object
knn_2 = KNeighborsClassifier()
#Use GridSearch
clf = GridSearchCV(knn_2, hyperparameters, cv=10)

In [ ]:
parameters_KNN = {
    'n_neighbors': (1,10, 1),
    'leaf_size': (20,40,1),
    'p': (1,2),
    'weights': ('uniform', 'distance'),
    'metric': ('minkowski', 'chebyshev'),
                   
# with GridSearch
grid_search_KNN = GridSearchCV(
    estimator=estimator_KNN,
    param_grid=parameters_KNN,
    scoring = 'accuracy',
    n_jobs = -1,
    cv = 5